In [77]:
!pip install keras_cv image_classifiers
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from classification_models.keras import Classifiers
from keras.datasets import cifar100
from keras import Sequential
from keras import Input
from keras.layers import UpSampling2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import RandomFlip
from keras.layers import RandomTranslation
from keras.layers import RandomRotation
from keras.layers import RandomBrightness
from keras.layers import RandomContrast
from keras.layers import RandomZoom
from keras.layers import GlobalAveragePooling2D
from keras_cv.layers import RandomShear
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Model
import keras_cv
import re
import os
import datetime

In [65]:
n_epoch = 1000 # Il y a le early stopping
batch_size = 400

In [66]:
(X_train, y_train), (X_test, y_test) = cifar100.load_data()
X_train = np.float64(X_train)/255
X_test = np.float64(X_test)/255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

num_classes = 100

In [67]:
modele_enseignant = tf.keras.models.load_model("/content/modele_745.keras")

In [68]:
y_train_enseignant = modele_enseignant.predict(X_train)

1563/1563 [==============================] - 27s 16ms/step


In [92]:
ResNet18, preprocess_input = Classifiers.get('resnet18')
resnet = ResNet18((224, 224, 3), weights='imagenet', include_top=False)
resnet_output = GlobalAveragePooling2D()(resnet.output)
resnet = Model(inputs=resnet.input, outputs=resnet_output)

for layer in resnet.layers[:]:
  if (re.match("^.*(_conv2|_bn*)$", layer.name)):
    layer.trainable = True
  else:
    layer.trainable = False

resnet.summary()

Model: "model_16"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 data (InputLayer)           [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 bn_data (BatchNormalizatio  (None, 224, 224, 3)          9         ['data[0][0]']                
 n)                                                                                               
                                                                                                  
 zero_padding2d_108 (ZeroPa  (None, 230, 230, 3)          0         ['bn_data[0][0]']             
 dding2D)                                                                                         
                                                                                           

In [95]:
model = Sequential([
    UpSampling2D(size=(7, 7)),
    resnet,
    Dense(256, activation="sigmoid"),
    Dropout(0.25),
    Dense(num_classes, activation="softmax")])

In [96]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train_enseignant, epochs=n_epoch, batch_size=batch_size, validation_split = 0.1,callbacks = early_stopping_callback)
model.save("/root/modele_enseigne.keras")

Epoch 1/1000
113/113 [==============================] - 17s 127ms/step - loss: 4.6252 - accuracy: 0.0109 - val_loss: 4.5974 - val_accuracy: 0.0160
Epoch 2/1000
 48/113 [===========>..................] - ETA: 7s - loss: 4.6004 - accuracy: 0.0137

KeyboardInterrupt: 